# Reconhecimento de entidades

Optuna será utilizado para _parameter tuning_

In [33]:
!pip install --quiet optuna

In [34]:
import pandas as pd
import tensorflow as tf
from collections import defaultdict
import numpy as np
from sklearn.model_selection import train_test_split
import optuna

O _dataset_ utilizado consiste de linhas contendo o número da frase ("_sentence\_number_"), seguido da palavra ("_word_"), que, por sua vez, é seguida de uma _tag_.

In [35]:
data = pd.read_csv('./notebooks/sentences.csv')
data.head()

,sentence_number,word,tag
0,1,como,OTHER
1,1,usar,OTHER
2,1,lucros,OTHER
3,1,e,OTHER
4,1,reinvestir,INVEST


São criados mapeamentos entre palavras e números distintos para cada. Isso também é feito para as tags.

In [36]:
vocab = set(data['word'])
vocab_list = list(vocab)

word2idx = {}
idx2word = {}

for idx, word in enumerate(vocab_list):
    word2idx[word] = idx
    idx2word[idx] = word

tags = set(data['tag'])
tags_list = list(tags)

tag2idx = {}
idx2tag = {}

for idx, tag in enumerate(tags_list):
    tag2idx[tag] = idx
    idx2tag[idx] = tag

Para os fins de treinamento, as palavras originais não têm serventia. Cria-se um novo dataset contendo somente os números correspondentes aos valores originais, de acordo com a conversão que acaba de ser feita.

In [37]:
converted_data = data.copy()

converted_data['word'] = converted_data['word'].transform(lambda word: word2idx[word])
converted_data['tag'] = converted_data['tag'].transform(lambda tag: tag2idx[tag])
converted_data.head()

,sentence_number,word,tag
0,1,63,0
1,1,85,0
2,1,39,0
3,1,19,0
4,1,104,1


Agregam-se em listas os valores de cada coluna de acordo com o número de cada frase.

In [38]:
list_data = converted_data.groupby(['sentence_number'])[['word', 'tag']].agg(lambda i: list(i))
list_data.head()

,word,tag
sentence_number,,
1,"[63, 85, 39, 19, 104, 91, 23, 48, 69, 42]","[0, 0, 0, 0, 1, 14, 0, 14, 0, 8]"
2,"[63, 10, 88, 25, 66, 1, 2, 54, 4, 121, 29, 79,...","[0, 0, 0, 17, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,"[57, 31, 84, 109, 68, 117]","[0, 0, 0, 0, 0, 0]"
4,"[31, 81, 87, 94, 35, 17, 95, 1]","[0, 0, 0, 0, 0, 11, 0, 2]"
5,"[107, 29, 35, 105, 110, 60, 52, 33, 45, 24, 93]","[0, 0, 0, 13, 0, 0, 0, 0, 1, 0, 0]"


Tomando-se uma lista de índices do _dataset_, e convertendo-os de volta em palavras, vê-se que a ordem é mantida

In [39]:
[idx2word[idx] for idx in list_data['word'][5]]

['quais',
 'são',
 'os',
 'melhores',
 'recursos',
 'para',
 'aprender',
 'sobre',
 'investimento',
 'em',
 'títulos']

O tamanho da maior frase será necessário

In [40]:
max_len = list_data['word'].map(len).max()
max_len

28

A função pad_sequences de `tf.keras.preprocessing.sequence` é útil para adicionar elementos extras a listas, fazendo com que tenham todas o mesmo tamanho.

`tf.keras.utils.to_categorical` é usada para converter a representação das tags de números para _arrays_ de _One-Hot-Encoding_.

In [41]:
pad_tokens = tf.keras.preprocessing.sequence.pad_sequences(list_data['word'])
pad_tags = tf.keras.preprocessing.sequence.pad_sequences(list_data['tag'])
pad_tags = [tf.keras.utils.to_categorical(tag, num_classes=len(tags)) for tag in pad_tags]

train_tokens, test_tokens, train_tags, test_tags = train_test_split(pad_tokens, pad_tags)

In [42]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Bidirectional
from tensorflow.keras import optimizers

Optuna será utilizado para executar um número de experimentos com modelos que receberão parâmetros "_embedding\_output\_dim_" e "_num\_epochs_" diferentes. "_embedding\_output\_dim_" dita o tamanho da saída da camada de _embedding_, e "_num\_epochs_" o número de épocas de treinamento.

In [43]:
input_dim = len(vocab)
input_length = max_len

def objective(trial):
    embedding_output_dim = trial.suggest_int("embedding_output_dim", 16, 256)

    model = Sequential([
        Embedding(input_dim=input_dim, output_dim=embedding_output_dim, input_length=input_length),
        Bidirectional(LSTM(units=embedding_output_dim, return_sequences=True), merge_mode='concat'),
        LSTM(units=embedding_output_dim, return_sequences=True),
        TimeDistributed(Dense(len(tags), activation='relu'))
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    num_epochs = trial.suggest_int("num_epochs", 5, 150)

    model.fit(train_tokens, np.array(train_tags), verbose=1, epochs=num_epochs)

    return model.evaluate(test_tokens, np.array(test_tags), return_dict=True)['accuracy']

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2021-08-21 13:50:08,086] A new study created in memory with name: no-name-c2581ee8-1805-4ba6-aaea-ff65cbba2f33


Epoch 1/121
1/1 [==============================] - 5s 5s/step - loss: 11.4977 - accuracy: 0.0020
Epoch 2/121
1/1 [==============================] - 0s 177ms/step - loss: 1.4087 - accuracy: 0.9067
Epoch 3/121
1/1 [==============================] - 0s 387ms/step - loss: 1.1520 - accuracy: 0.9067
Epoch 4/121
1/1 [==============================] - 0s 197ms/step - loss: 0.8501 - accuracy: 0.9067
Epoch 5/121
1/1 [==============================] - 0s 393ms/step - loss: 0.7830 - accuracy: 0.9067
Epoch 6/121
1/1 [==============================] - 0s 274ms/step - loss: 0.7536 - accuracy: 0.9067
Epoch 7/121
1/1 [==============================] - 0s 137ms/step - loss: 0.7340 - accuracy: 0.9067
Epoch 8/121
1/1 [==============================] - 0s 123ms/step - loss: 0.7221 - accuracy: 0.9067
Epoch 9/121
1/1 [==============================] - 0s 296ms/step - loss: 0.7101 - accuracy: 0.9067
Epoch 10/121
1/1 [==============================] - 0s 233ms/step - loss: 0.6903 - accuracy: 0.9067
Epoch 11/12

[I 2021-08-21 13:50:40,086] Trial 0 finished with value: 0.9234693646430969 and parameters: {'embedding_output_dim': 191, 'num_epochs': 121}. Best is trial 0 with value: 0.9234693646430969.


Epoch 1/23
1/1 [==============================] - 5s 5s/step - loss: 3.4190 - accuracy: 0.1984
Epoch 2/23
1/1 [==============================] - 0s 430ms/step - loss: 1.2884 - accuracy: 0.9067
Epoch 3/23
1/1 [==============================] - 0s 312ms/step - loss: 1.1813 - accuracy: 0.9067
Epoch 4/23
1/1 [==============================] - 0s 330ms/step - loss: 1.2140 - accuracy: 0.9048
Epoch 5/23
1/1 [==============================] - 0s 369ms/step - loss: 1.1731 - accuracy: 0.9028
Epoch 6/23
1/1 [==============================] - 0s 337ms/step - loss: 1.1056 - accuracy: 0.9048
Epoch 7/23
1/1 [==============================] - 0s 433ms/step - loss: 1.0596 - accuracy: 0.9067
Epoch 8/23
1/1 [==============================] - 0s 349ms/step - loss: 1.0134 - accuracy: 0.9067
Epoch 9/23
1/1 [==============================] - 0s 410ms/step - loss: 0.9844 - accuracy: 0.9067
Epoch 10/23
1/1 [==============================] - 1s 504ms/step - loss: 1.1330 - accuracy: 0.9067
Epoch 11/23
1/1 [=====

[I 2021-08-21 13:50:55,917] Trial 1 finished with value: 0.9234693646430969 and parameters: {'embedding_output_dim': 246, 'num_epochs': 23}. Best is trial 0 with value: 0.9234693646430969.


...

[I 2021-08-21 14:11:24,481] Trial 47 finished with value: 0.9234693646430969 and parameters: {'embedding_output_dim': 200, 'num_epochs': 117}. Best is trial 0 with value: 0.9234693646430969.


Epoch 1/5
1/1 [==============================] - 6s 6s/step - loss: 10.8636 - accuracy: 0.0179
Epoch 2/5
1/1 [==============================] - 0s 116ms/step - loss: 1.4402 - accuracy: 0.9067
Epoch 3/5
1/1 [==============================] - 0s 161ms/step - loss: 1.2941 - accuracy: 0.9067
Epoch 4/5
1/1 [==============================] - 0s 191ms/step - loss: 1.1611 - accuracy: 0.9067
Epoch 5/5
1/1 [==============================] - 2s 2s/step - loss: 1.3289 - accuracy: 0.9235


[I 2021-08-21 14:11:32,984] Trial 48 finished with value: 0.9234693646430969 and parameters: {'embedding_output_dim': 90, 'num_epochs': 5}. Best is trial 0 with value: 0.9234693646430969.


Epoch 1/125
1/1 [==============================] - 5s 5s/step - loss: 6.9950 - accuracy: 0.0020
Epoch 2/125
1/1 [==============================] - 0s 170ms/step - loss: 1.6899 - accuracy: 0.9067
Epoch 3/125
1/1 [==============================] - 0s 196ms/step - loss: 1.3682 - accuracy: 0.9067
Epoch 4/125
1/1 [==============================] - 0s 156ms/step - loss: 1.1693 - accuracy: 0.9067
Epoch 5/125
1/1 [==============================] - 0s 166ms/step - loss: 0.9950 - accuracy: 0.9067
Epoch 6/125
1/1 [==============================] - 0s 137ms/step - loss: 0.8828 - accuracy: 0.9067
Epoch 7/125
1/1 [==============================] - 0s 234ms/step - loss: 0.8217 - accuracy: 0.9067
Epoch 8/125
1/1 [==============================] - 0s 190ms/step - loss: 0.7866 - accuracy: 0.9067
Epoch 9/125
1/1 [==============================] - 0s 186ms/step - loss: 0.7530 - accuracy: 0.9067
Epoch 10/125
1/1 [==============================] - 0s 189ms/step - loss: 0.7197 - accuracy: 0.9067
Epoch 11/125

[I 2021-08-21 14:12:01,573] Trial 49 finished with value: 0.9234693646430969 and parameters: {'embedding_output_dim': 77, 'num_epochs': 125}. Best is trial 0 with value: 0.9234693646430969.


In [44]:
input_dim = len(vocab)
input_length = max_len

best_trial_params = study.best_trial.params

embedding_output_dim = best_trial_params['embedding_output_dim']
num_epochs = best_trial_params['num_epochs']

model = Sequential([
    Embedding(input_dim=input_dim, output_dim=embedding_output_dim, input_length=input_length),
    Bidirectional(LSTM(units=embedding_output_dim, return_sequences=True, dropout=0.01), merge_mode='concat'),
    LSTM(units=embedding_output_dim, return_sequences=True, dropout=0.01),
    TimeDistributed(Dense(len(tags), activation='relu'))
])

optimizer = optimizers.Adam(clipvalue=0.5)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.fit(train_tokens, np.array(train_tags), epochs=num_epochs)

model.evaluate(test_tokens, np.array(test_tags))

Epoch 1/121
1/1 [==============================] - 6s 6s/step - loss: 5.8709 - accuracy: 0.4127
Epoch 2/121
1/1 [==============================] - 0s 146ms/step - loss: 1.5083 - accuracy: 0.9067
Epoch 3/121
1/1 [==============================] - 0s 154ms/step - loss: 1.1873 - accuracy: 0.9067
Epoch 4/121
1/1 [==============================] - 0s 159ms/step - loss: 1.1211 - accuracy: 0.9067
Epoch 5/121
1/1 [==============================] - 0s 155ms/step - loss: 1.1016 - accuracy: 0.9067
Epoch 6/121
1/1 [==============================] - 0s 141ms/step - loss: 1.0723 - accuracy: 0.9067
Epoch 7/121
1/1 [==============================] - 0s 125ms/step - loss: 1.1142 - accuracy: 0.9067
Epoch 8/121
1/1 [==============================] - 0s 125ms/step - loss: 1.1959 - accuracy: 0.9067
Epoch 9/121
1/1 [==============================] - 0s 126ms/step - loss: 1.2290 - accuracy: 0.9067
Epoch 10/121
1/1 [==============================] - 0s 116ms/step - loss: 1.2577 - accuracy: 0.9067
Epoch 11/121

[nan, 0.9234693646430969]

In [45]:
t = 'como usar lucros e reinvestir sem ser esmagado por impostos'.split(' ')
t2 = [word2idx[word] for word in t if word in vocab]
t3 = model.predict(t2)
[idx2tag[np.argmax(cat)] for cat in t3]

['OTHER',
 'OTHER',
 'OTHER',
 'OTHER',
 'OTHER',
 'OTHER',
 'OTHER',
 'OTHER',
 'OTHER',
 'OTHER']

Não foram obtidos bons resultados dada a pequeneza do _dataset_, mas é possível observar que a rede neural aprendeu a sugerir a tag _OTHER_, que possui número igual ao usado para _padding_.